# Introduction to GPU Programming with Python
## Solutions to the exercises

### Exercise 0
Matrix multiplication using jit decorator (CPU)

In [ ]:
import numpy as np
from numba import jit,njit
from numba import vectorize,float64
from numba import prange

In [ ]:
@jit('void(float64[:,:],float64[:,:],float64[:,:])',parallel=True)
def matmul(A,B,C):
    # iterating by row of A
    for i in prange(len(A)):
  
        # iterating by coloum by B 
        for j in prange(len(B[0])):
  
            # iterating by rows of B
            for k in range(len(B)):
                C[i][j] += A[i][k] * B[k][j]

In [ ]:
A=np.random.rand(128,128)
B=np.random.rand(128,128)
C=np.zeros(shape=(128,128))

In [ ]:
%timeit matmul(A,B,C)

### Exercise 1
Lets do the following exercise where each element of an array is incremented : array[i] = array[i] + 1

In [ ]:
import numpy
from numba import cuda

In [ ]:
@cuda.jit
def kernel1(array):
    pos = cuda.grid(1)
    if pos<array.size:
        array[pos] += 1

In [ ]:
data=numpy.ones(12800,dtype=np.int32)
threads=32
blocks = (data.size + (threads - 1)) // threads
print(blocks)

In [ ]:
# Run the kernel and measure execution time:
%timeit kernel1[blocks,threads](data)
print(data)

In [ ]:
# Take advatage of excplicit data management and copy an array to GPU before kernel execution. 
# Then measure the execution time again
d_data = cuda.to_device(data)
%timeit kernel1[blocks,threads](d_data)

### Exercice 2
Integer array, sent to GPU where its indices are reversed, i.e. array[0]=array[N-1], array[1]=array[N-2], etc

In [ ]:
# Import required libs
import numpy as np
from numba import cuda, float32

In [ ]:
#Part 3: Create a CUDA kernel with @cuda.jit decorator
# Kernel: reverse the array content using appropriate indices. 
# To do so you may need input and output indices. Implement kernel with possibility of multiple thread blocks.
@cuda.jit
def reverseArrayBlock(d_out,d_in):
    ind_in = cuda.grid(1) ## Index of the current thread
    ind_out = cuda.gridsize(1)-ind_in-1 ## Total number of threads - in -1
    if ind_in<d_in.size:
        d_out[ind_out] = d_in[ind_in]

In [ ]:
# Define CUDA grid
dim=256
NumBlocks=1
NumThreadsPerBlock=dim

In [ ]:
#Part 1: Create arrays on CPU and GPU (if you want to)
a = np.arange(0,dim,dtype=np.int32)
b = np.zeros(dim,dtype=np.int32)

In [ ]:
#Part 2: Initialize host array
# Already initialized

In [ ]:
#Part 4: Call the kernel function
%timeit reverseArrayBlock[NumBlocks,NumThreadsPerBlock](b,a)

In [ ]:
#Part 5: Verify the result
print(b)

In [ ]:
#Part 5: Take advantage of explicit data management
d_a = cuda.to_device(a)
d_b = cuda.device_array_like(b)
%timeit reverseArrayBlock[NumBlocks,NumThreadsPerBlock](d_b,d_a)
d_b.copy_to_host(b)
print(b)

### Exercice 3
Repeat Excercise 2 with multiple blocks per CUDA grid (NumBlocks > 1)

In [ ]:
# Import required libs
import numpy as np
from numba import cuda, float32

In [ ]:
#Part 3: Create a CUDA kernel with @cuda.jit decorator
# Kernel: reverse the array content using appropriate indices. 
# To do so you may need input and output indices. Implement kernel with possibility of multiple thread blocks.
@cuda.jit
def reverseArrayBlock(d_out,d_in):
    ind_in = cuda.blockDim.x*cuda.blockIdx.x + cuda.threadIdx.x; ## Index of the current thread
    ind_out = cuda.gridsize(1)-ind_in-1 ## Total number of threads - in -1
    if ind_in<d_in.size:
        d_out[ind_out] = d_in[ind_in]

In [ ]:
# Define CUDA grid
dim=256*1000
NumThreadsPerBlock=128
NumBlocks = (dim + (NumThreadsPerBlock - 1)) // NumThreadsPerBlock
print(NumBlocks)

In [ ]:
#Part 1: Create arrays on CPU and GPU (if you want to)
a = np.arange(0,dim,dtype=np.int32)
b = np.zeros(dim,dtype=np.int32)

In [ ]:
#Part 2: Initialize host array
# Already initialized

In [ ]:
#Part 4: Call the kernel function
%timeit reverseArrayBlock[NumBlocks,NumThreadsPerBlock](b,a)

In [ ]:
#Part 5: Verify the result
print(b)

In [ ]:
#Part 5: Take advantage of explicit data management
d_a = cuda.to_device(a)
d_b = cuda.device_array_like(b)
%timeit reverseArrayBlock[NumBlocks,NumThreadsPerBlock](d_b,d_a)
b = d_b.copy_to_host()
print(b)

### Exercise 4
Polynomial evaluation on both GPU and CPU

In [ ]:
#Part 3: Modify polynomial function to make it work with numba.cuda
@cuda.jit
def cuda_polyval(result, array, coeffs):
    # Evaluate a polynomial function over an array with Horner's method.
    # The coefficients are given in descending order.
    i = cuda.grid(1) # equivalent to i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    val = coeffs[0]
    for coeff in coeffs[1:]:
        val = val * array[i] + coeff
    result[i] = val

In [ ]:
#Part 1: Allocate integer array (int32), size of 2048 * 1024. Also make an empty array for result, same size
array = np.random.randn(2048 * 1024).astype(np.float32)
coeffs = np.float32(range(1, 10))
result = np.empty_like(array)

In [ ]:
#Part 2: Prepare grid
blocks=2048
threads=1024

In [ ]:
#Part 4: Call the kernel and measure execution time
%timeit cuda_polyval[blocks,threads](result, array, coeffs)

In [ ]:
#Part 5: Call the built-in NumPy polynomial function  np.polyval(coeffs, array) and compare results
numpy_result = np.polyval(coeffs, array)
print('Maximum relative error compared to numpy.polyval:', np.max(np.abs(numpy_result - result)))

In [ ]:
#Part 6: Go back to the kernel (Part 3) and modify it to make it work on CPU with @jit
@jit
def host_polyval_CPU(result, array, coeffs):
    for i in range(len(array)):
        val = coeffs[0]
        for coeff in coeffs[1:]:
            val = val * array[i] + coeff
        result[i] = val

In [ ]:
%timeit host_polyval_CPU(result, array, coeffs)
print('Maximum relative error compared to numpy.polyval:', np.max(np.abs(numpy_result - result)))

### Exercise 5
Matrix multiplication WITH GLOBAL MEMORY

In [ ]:
def matmul(A,B,C):
    # iterating by row of A
    for i in range(len(A)):
  
        # iterating by coloum by B 
        for j in range(len(B[0])):
  
            # iterating by rows of B
            for k in range(len(B)):
                C[i][j] += A[i][k] * B[k][j]

In [ ]:
#Part 1: Create matrices A,B,C as numpy arrays. Fill A and B with random numbers.
A=np.random.rand(128,128)
B=np.random.rand(128,128)
C=np.zeros(shape=(128,128))

In [ ]:
#Part 2: Calculate number of blocks and threads
threads=32
blocks = C.shape[0]*C.shape[1]//threads +1
blockdim = (threads,threads)
griddim = (blocks,blocks)

In [ ]:
#Part 3: Create a CUDA kernel with @cuda.jit decorator
@cuda.jit
def matmul(A,B,C):
    i,j=cuda.grid(2)
    if i<C.shape[0] and j<C.shape[1]:
        tmp=0.0
        for k in range(A.shape[1]):
            tmp+=A[i,k]*B[k,j]
        C[i,j]=tmp

In [ ]:
#Part 4: Call the kernel function and time it to get the execution time
%timeit matmul[griddim,blockdim](A,B,C)

In [ ]:
#Part 5: Create A,B,C manually on the GPU and copy data to the GPU arrays
d_A=cuda.to_device(A)
d_B=cuda.to_device(B)
d_C=cuda.to_device(C)

In [ ]:
#Part 6: Call the kernel function and time it to get the execution time. Compare the execution times.
%timeit matmul[blocks,threads](d_A,d_B,d_C)

### Mandelbrot Example

In [ ]:
import numpy as np
from matplotlib.pyplot import imshow, show
from timeit import default_timer as timer
from numba import jit,cuda

In [ ]:
def mandel(x, y, max_iters):
  
  c = complex(x, y)
  z = 0.0j
  for i in range(max_iters):
    z = z*z + c
    if (z.real*z.real + z.imag*z.imag) >= 4:
      return i

  return max_iters

In [ ]:
#Part1 : Make a create_fractal function
def create_fractal(min_x, max_x, min_y, max_y, image, iters):
  height = image.shape[0]
  width = image.shape[1]

  pixel_size_x = (max_x - min_x) / width
  pixel_size_y = (max_y - min_y) / height
    
  for x in range(width):
    real = min_x + x * pixel_size_x
    for y in range(height):
      imag = min_y + y * pixel_size_y
      color = mandel(real, imag, iters)
      image[y, x] = color


In [ ]:
#Part 2: Next we create an empty array, size 1024x1024, type np.uint8. Call create_fractal with appropriate coordinates 
#to fit the whole mandelbrot set. Then show the image. Measure the execution time.
image = np.zeros((1024, 1024), dtype = np.uint8)
%timeit create_fractal(-2.0, 1.0, -1.0, 1.0, image, 20) 

imshow(image)
show()

In [ ]:
#Part 3: Modify both mandel and create_fractal function and optimize/parallelize them with jit decorator 
#to work on the CPU
@jit
def mandel(x, y, max_iters):
  
  c = complex(x, y)
  z = 0.0j
  for i in range(max_iters):
    z = z*z + c
    if (z.real*z.real + z.imag*z.imag) >= 4:
      return i

  return max_iters

@jit
def create_fractal(min_x, max_x, min_y, max_y, image, iters):
  height = image.shape[0]
  width = image.shape[1]

  pixel_size_x = (max_x - min_x) / width
  pixel_size_y = (max_y - min_y) / height
    
  for x in range(width):
    real = min_x + x * pixel_size_x
    for y in range(height):
      imag = min_y + y * pixel_size_y
      color = mandel(real, imag, iters)
      image[y, x] = color

In [ ]:
#Part 4: Run again and measure the execution time
image = np.zeros((1024, 1024), dtype = np.uint8)
%timeit create_fractal(-2.0, 1.0, -1.0, 1.0, image, 20) 

imshow(image)
show()

In [ ]:
#Part 5: Write the kernel function mandel_kernel  with numba.cuda. Also modify mandel to mandel_gpu with cuda.jit
mandel_gpu = cuda.jit(device=True)(mandel)

@cuda.jit
def mandel_kernel(min_x, max_x, min_y, max_y, image, iters):
  height = image.shape[0]
  width = image.shape[1]

  pixel_size_x = (max_x - min_x) / width
  pixel_size_y = (max_y - min_y) / height

  startX, startY = cuda.grid(2)
  gridX = cuda.gridDim.x * cuda.blockDim.x;
  gridY = cuda.gridDim.y * cuda.blockDim.y;

  for x in range(startX, width, gridX):
    real = min_x + x * pixel_size_x
    for y in range(startY, height, gridY):
      imag = min_y + y * pixel_size_y 
      image[y, x] = mandel_gpu(real, imag, iters)

In [ ]:
#Part 6: Create cuda grid
image = np.zeros((1024, 1024), dtype = np.uint8)
blockdim = (32,8)
griddim = (32,16)

In [ ]:
#Part 7: Run the kernel. Also measure the execution time.
%timeit mandel_kernel[griddim,blockdim](-2.0, 1.0, -1.0, 1.0, image, 20) 

### Matrix multiplication WITH SHARED MEMORY

In [ ]:
import numpy as np
from numba import cuda, float32

In [ ]:
#Part 3: Create a CUDA kernel with @cuda.jit decorator

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16
@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    
    # Define global and thread indices
    x, y = cuda.grid(2)
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    # Define number of blocks per grid
    bpg = cuda.gridDim.x    # blocks per grid
    
    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return
    
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]
        
        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    # Put tmp into C matrix
    C[x, y] = tmp

In [ ]:
#Part 1: Create matrices A,B,C as numpy arrays (size 128x128,float32). Fill A and B with random numbers.
N=128
A=np.random.rand(N,N).astype(np.float32)
B=np.random.rand(N,N).astype(np.float32)
C=np.zeros(shape=(N,N)).astype(np.float32)

In [ ]:
#Part 2: Calculate number of blocks and threads
griddim = (N//TPB,N//TPB)
blockdim = (TPB,TPB)

In [ ]:
#Part 4: Call the kernel function and time it to get the execution time
%timeit fast_matmul[griddim,blockdim](A, B, C)